In [78]:
# datasets from my GitHub
!git clone https://github.com/William-HTP/Google-Play-App-Reviews-Datasets.git

Cloning into 'Google-Play-App-Reviews-Datasets'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), 3.18 MiB | 5.98 MiB/s, done.


In [1]:
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix

In [87]:
# load dataset
df = pd.read_excel('/content/Google-Play-App-Reviews-Datasets/dataset_balanced_8000.xlsx')
pd.set_option('display.max_colwidth', None)
df

,ID,review,label,new_label,remove_all_stopwords,remove_some_stopwords,stemming,lemmatization
0,1,Not able to add freinds. It show something went wrong.. Recommend not to install it,bug report,0,able add freinds show something went wrong recommend install,not able add freinds show something went wrong recommend not install,not abl to add freind it show someth went wrong recommend not to instal it,not able to add freinds it show something went wrong recommend not to install it
1,2,Calls keep dropping for no reason and is super annoying,bug report,0,calls keep dropping reason super annoying,calls keep dropping no reason super annoying,call keep drop for no reason and is super annoy,call keep dropping for no reason and is super annoying
2,3,Can't access to my account solve this issue immediately,bug report,0,cant access account solve issue immediately,cant access account solve issue immediately,cant access to my account solv thi issu immedi,cant access to my account solve this issue immediately
3,4,"I don't know what's wrong with my own WhatsApp. Its not working. And if I want to sign up, it will just be bringing try again later. You guys should see to it.",bug report,0,dont know whats wrong whatsapp working want sign bringing try later guys see,dont know whats wrong whatsapp not working want sign up will bringing try again later guys should see,i dont know what wrong with my own whatsapp it not work and if i want to sign up it will just be bring tri again later you guy should see to it,i dont know whats wrong with my own whatsapp it not working and if i want to sign up it will just be bringing try again later you guy should see to it
4,5,Cannot record audio while taking video,bug report,0,cannot record audio taking video,cannot record audio taking video,cannot record audio while take video,cannot record audio while taking video
...,...,...,...,...,...,...,...,...
7995,7996,"good app, l will give you 5 star!",rating,2,good app l give 5 star,good app l will give 5 star,good app l will give you 5 star,good app l will give you 5 star
7996,7997,Never had a problem with it.,rating,2,never problem,never problem,never had a problem with it,never had a problem with it
7997,7998,It s great and I recommend it all the time to others.,rating,2,great recommend time others,great recommend all time others,it s great and i recommend it all the time to other,it s great and i recommend it all the time to others
7998,7999,"Everyone can use this app ,",rating,2,everyone use app,everyone can use app,everyon can use thi app,everyone can use this app


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     8000 non-null   int64 
 1   review                 8000 non-null   object
 2   label                  8000 non-null   object
 3   new_label              8000 non-null   int64 
 4   remove_all_stopwords   8000 non-null   object
 5   remove_some_stopwords  8000 non-null   object
 6   stemming               8000 non-null   object
 7   lemmatization          8000 non-null   object
dtypes: int64(2), object(6)
memory usage: 500.1+ KB


In [89]:
# dataset label
df['label'].value_counts()

bug report         2000
feature request    2000
rating             2000
user experience    2000
Name: label, dtype: int64

In [90]:
# key words in each labels
bug_reports = ["bug", "fix", "problem", "issue", "defect", "crash", "solve"]
feature_requests = ["add", "please", "could", "would", "hope", "improve", "miss", "need", "prefer", "request", "should", "suggest", "want", "wish"]
user_experiences = ["help", "support", "assist", "when", "situation"]
ratings = ["great", "good", "nice", "very", "cool", "love", "hate", "bad", "worst"]

In [91]:
# list of true_labels and review
true_labels = df['label'].values
review = df['review'].values
# review = df['remove_all_stopwords'].values
# review = df['remove_some_stopwords'].values
# review = df['stemming'].values
# review = df['lemmatization'].values

In [92]:
count = 0
pred_labels = []

for i in range(len(review)):
    string_mark = {"bug report":0, "feature request":0, "user experience":0, "rating":0}
    sent = review[i]
    sent = sent.lower()
    token = sent.split(" ")

    # keywords mathching
    for j in token:
        if j in bug_reports:
            string_mark["bug report"] += 1
        if j in feature_requests:
            string_mark["feature request"] += 1
        if j in user_experiences:
            string_mark["user experience"] += 1
        if j in ratings:
            string_mark["rating"] += 1

    # predict label
    if max(string_mark.values()) != 0:
        label = max(string_mark,key=lambda x:string_mark[x])
        pred_labels.append(label)
        if label == true_labels[i]:
            count += 1
    else:
        pred_labels.append("NO label")
        count += 0

# accuracy
print('Accuracy: {:.2%}'.format(count/len(review)))

Accuracy: 30.81%


In [93]:
# classification_report
print(classification_report(true_labels, pred_labels))

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

       NO label       0.00      0.00      0.00         0
     bug report       0.70      0.24      0.36      2000
feature request       0.57      0.56      0.56      2000
         rating       0.40      0.36      0.38      2000
user experience       0.33      0.07      0.12      2000

       accuracy                           0.31      8000
      macro avg       0.40      0.25      0.28      8000
   weighted avg       0.50      0.31      0.35      8000



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
